In [5]:
import sys
import os

# Go two levels up from the notebook to the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))

# Join the path to 'src'
src_path = os.path.join(project_root, "src")

# Add 'src' to Python path
if src_path not in sys.path:
    sys.path.append(src_path)

# Confirm it's added
print("src path added:", src_path)


src path added: c:\Users\ABC\Desktop\10Acadamy\week_7\Shipping-a-Data-Product\src


In [6]:
# 2️⃣ Load .env and set PYTHONPATH
import os, sys
from dotenv import load_dotenv
from pathlib import Path

project_root = Path().cwd().parent / "Shipping-a-Data-Product"
load_dotenv(dotenv_path=project_root / ".env")
sys.path.append(str(project_root / "src"))


In [7]:
# 3️⃣ Run raw_loader to ingest JSON & images
from raw_loader import load_messages, load_images

# Load yesterday’s (or today’s) partitions
load_messages(date_str="2025-07-13")
load_images(date_str="2025-07-13")


In [8]:
# 4️⃣ Install & run dbt to build staging + marts
%env DBT_PROFILES_DIR=./dbt_project
!dbt deps   # fetches any packages
!dbt seed   # if you have any seeds
!dbt run    # builds models in analytics schema
!dbt test   # runs built-in tests


env: DBT_PROFILES_DIR=./dbt_project


Usage: dbt deps [OPTIONS]
Try 'dbt deps -h' for help.

Error: Got unexpected extra arguments (# fetches any packages)
Usage: dbt seed [OPTIONS]
Try 'dbt seed -h' for help.

Error: Invalid value for '--profiles-dir': Path './dbt_project' does not exist.
Usage: dbt run [OPTIONS]
Try 'dbt run -h' for help.

Error: Invalid value for '--profiles-dir': Path './dbt_project' does not exist.
Usage: dbt test [OPTIONS]
Try 'dbt test -h' for help.

Error: Invalid value for '--profiles-dir': Path './dbt_project' does not exist.
